# Logistic Regression
Categorical data - Classification issue on unbounded values (0 or 1)
</br>Requires the use of a *sigmoid activation function*
</br>Sigmoid function maps between 0 and 1: $s(z) = \frac{1}{1 + e^{-z}}$, where $z = w^tx + b$

- $z$: linear combination
- $w$: weight vector
- $b$: bias
- $w^tx$: dot product of the weights and features
- $s(z) or \hat{y}$: predicted probability

Rule:
- If s(z) >= 0.5 then -> 1
- Else, predict 0

## Cost function

Unlike Linear Regression, we cannot use mean squared error — applying it to sigmoid outputs leads to a **non-convex cost surface**, making optimization unstable.

Instead, logistic regression uses **binary cross-entropy loss** (also known as **log loss**):

**Loss =**  
`-(1/m) * Σ [y(i) * log(ŷ(i)) + (1 - y(i)) * log(1 - ŷ(i))]`

Where:  
- `y(i)` is the true label (0 or 1)  
- `ŷ(i)` is the predicted probability from the sigmoid function

---

### 🔍 Intuition

- The loss measures how far off the predicted probability `ŷ` is from the actual label `y`.
- If the **true label is 1**, only the term `y * log(ŷ)` matters:
  - If `ŷ` is close to 1 → **low loss**
  - If `ŷ` is close to 0 → **high loss**
- If the **true label is 0**, only the term `(1 - y) * log(1 - ŷ)` matters:
  - If `ŷ` is close to 0 → **low loss**
  - If `ŷ` is close to 1 → **high loss**
- This means **confident, wrong predictions are penalized heavily**, which encourages the model to output well-calibrated probabilities.

---

### 🎯 Why is this loss function used?

- It follows from **maximum likelihood estimation** — we're maximizing the probability that our model assigns to the correct labels.
- It is **convex** when used with a sigmoid, so gradient descent can reliably find a **global minimum**.



In [31]:
# Imports
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import pprint
import pickle

## Data Loading and Analysis

In [32]:
# From https://developer.ibm.com/articles/implementing-logistic-regression-from-scratch-in-python/
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer

In [33]:
data = load_breast_cancer()

In [34]:
# Combine features and target into a single DataFrame
df = pd.DataFrame(data.data, columns=data.feature_names)
df['diagnosis'] = data.target

# Split into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [35]:
df.sample(10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,diagnosis
54,15.10,22.02,97.26,712.8,0.09056,0.07081,0.052530,0.033340,0.1616,0.05684,...,31.69,117.70,1030.0,0.13890,0.2057,0.271200,0.15300,0.2675,0.07873,0
247,12.89,14.11,84.95,512.2,0.08760,0.13460,0.137400,0.039800,0.1596,0.06409,...,17.70,105.00,639.1,0.12540,0.5849,0.772700,0.15610,0.2639,0.11780,1
423,13.66,19.13,89.46,575.3,0.09057,0.11470,0.096570,0.048120,0.1848,0.06181,...,25.50,101.40,708.8,0.11470,0.3167,0.366000,0.14070,0.2744,0.08839,1
167,16.78,18.80,109.30,886.3,0.08865,0.09182,0.084220,0.065760,0.1893,0.05534,...,26.30,130.70,1260.0,0.11680,0.2119,0.231800,0.14740,0.2810,0.07228,0
257,15.32,17.27,103.20,713.3,0.13350,0.22840,0.244800,0.124200,0.2398,0.07596,...,22.66,119.80,928.8,0.17650,0.4503,0.442900,0.22290,0.3258,0.11910,0
353,15.08,25.74,98.00,716.6,0.10240,0.09769,0.123500,0.065530,0.1647,0.06464,...,33.22,121.20,1050.0,0.16600,0.2356,0.402900,0.15260,0.2654,0.09438,0
327,12.03,17.93,76.09,446.0,0.07683,0.03892,0.001546,0.005592,0.1382,0.06070,...,22.25,82.74,523.4,0.10130,0.0739,0.007732,0.02796,0.2171,0.07037,1
401,11.93,10.91,76.14,442.7,0.08872,0.05242,0.026060,0.017960,0.1601,0.05541,...,20.14,87.64,589.5,0.13740,0.1575,0.151400,0.06876,0.2460,0.07262,1
531,11.67,20.02,75.21,416.2,0.10160,0.09453,0.042000,0.021570,0.1859,0.06461,...,28.81,87.00,550.6,0.15500,0.2964,0.275800,0.08120,0.3206,0.08950,1
284,12.89,15.70,84.08,516.6,0.07818,0.09580,0.111500,0.033900,0.1432,0.05935,...,19.69,92.12,595.6,0.09926,0.2317,0.334400,0.10170,0.1999,0.07127,1


In [36]:
# Name of all parameters
list(df.columns.values)

['mean radius',
 'mean texture',
 'mean perimeter',
 'mean area',
 'mean smoothness',
 'mean compactness',
 'mean concavity',
 'mean concave points',
 'mean symmetry',
 'mean fractal dimension',
 'radius error',
 'texture error',
 'perimeter error',
 'area error',
 'smoothness error',
 'compactness error',
 'concavity error',
 'concave points error',
 'symmetry error',
 'fractal dimension error',
 'worst radius',
 'worst texture',
 'worst perimeter',
 'worst area',
 'worst smoothness',
 'worst compactness',
 'worst concavity',
 'worst concave points',
 'worst symmetry',
 'worst fractal dimension',
 'diagnosis']

### NOTE: 0 - Malignant, 1 - Benign

In [37]:
px.histogram(data_frame=df, x='diagnosis', color='diagnosis',color_discrete_sequence=['#05445E','#75E6DA'])

In [38]:
px.histogram(data_frame=df,x='mean area',color='diagnosis',color_discrete_sequence=['#05445E','#75E6DA'])

Now that the stock standard parameters have been shown, my guess is that most metrics will be different between malignant and benign types.

In [39]:
px.scatter(data_frame=df,x='worst area',color='diagnosis',color_discrete_sequence=['#05445E','#75E6DA'])

In [40]:
px.scatter(data_frame=df,x='mean fractal dimension',color='diagnosis',color_discrete_sequence=['#05445E','#75E6DA'])

Interestingly this one has less deviation between the two diagnoses. Google quotes it as: 'A tumour fractal dimension is a numerical value that quantifies the complexity and irregularity of a tumor's structure.' Therefore the complexity of the growth *does not infer* how dangerous it is.